In [ ]:
# !featurize dataset download 6878dbe0-2e3d-4065-92e0-1db6ec358071

In [ ]:
# pip3 install --user torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113 --force-reinstall --upgrade

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision.models import resnet50, convnext_tiny, convnext_base


import utils
import models
import label_processor

In [3]:
# DATA_PATH = '/home/featurize/data/medium/'
DATA_PATH = './dataset/medium/'
embed = label_processor.CStdLib(single=True)
CLASS_NUM = embed.class_num

# TODO: Ablation experiment (Mask)
# TODO: Use other instance segmentation models (e.g. Swin)
dataset = utils.ColorfulClothesCLF(DATA_PATH, class_num=CLASS_NUM, embed=embed, train=True)
data_distribution = dataset.clean_and_analyse()
print(data_distribution)
small_len = int(len(dataset) * 1.0)
small, _ = random_split(dataset, [small_len, len(dataset) - small_len])
val_len = int(len(small) * 0.1)
train, val = random_split(small, [len(small) - val_len, val_len])

train_loader = DataLoader(train, batch_size=64, shuffle=True, num_workers=6)
test_loader = DataLoader(val, batch_size=128, shuffle=False, num_workers=6)

tensor([29621, 16813, 12240, 10439,  8764,  8584,  6302,  5854,  5255,  3718,
         2031,  2701,  1415,  2044,   783,   208,   163,   182,   154,  5243,
         2619,  2214,  1737,  1775,  1448,   994,  2014,   997,   733,   757,
          745,   634,   612,   753,   460,   470,   517,   468,   488,   349,
          370,   535,   235,   203,   196,   191,   126,   140,   117,   113,
          263,    94,    49,    63,    76,    14,    35,   127,   205])


In [4]:
model = convnext_tiny(pretrained=True)
num_params = sum(p.numel() for p in model.parameters())
print("Number of parameters: {}".format(num_params))
print(model)

Number of parameters: 28589128
ConvNeXt(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm2d((96,), eps=1e-06, elementwise_affine=True)
    )
    (1): Sequential(
      (0): CNBlock(
        (block): Sequential(
          (0): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (1): Permute()
          (2): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in_features=96, out_features=384, bias=True)
          (4): GELU()
          (5): Linear(in_features=384, out_features=96, bias=True)
          (6): Permute()
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): CNBlock(
        (block): Sequential(
          (0): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (1): Permute()
          (2): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          (3): Linear(i

In [ ]:
model.fc = nn.Linear(768, CLASS_NUM)

In [ ]:
loss_weight = (1. / data_distribution).to(torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'))
# TODO: Ablation experiment
criterion = nn.CrossEntropyLoss(reduction='mean', weight=loss_weight)
# TODO: Add an introductory task: predict the RGB value.
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
trainer = utils.Trainer(model, class_num=CLASS_NUM, criterion=criterion, optimizer=optimizer, milestones=[2, 5, 10])

In [ ]:
trainer.train(train_loader=train_loader, test_loader=test_loader, epochs=20)